Input the name and the url of the product that you want to track in the `products` dictionary.

NOTE: For stopping the program, press 'ctrl' 'm' and 'i' at the SAME time to keep the output intact

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
from IPython.display import display, clear_output
from IPython.display import Audio, display


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def data_extractor(soup):
    try:
        price_data = soup.find("span", class_ = "a-price-whole")
        price = int("".join(price_data.text[:-1].split(",")))

        # price_data = soup.find("div", id = "corePriceDisplay_desktop_feature_div")

        # mrp_data = price_data.find("span", class_ = "a-size-small aok-offscreen".split())
        # mrp_str = mrp_data.find("span", class_ = "a-offscreen")
        # mrp = int("".join(mrp_str.text[1:].split(",")))

        disc = soup.find("span", class_ = "a-size-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage")
        discount = int(disc.text[1:-1])
    except AttributeError:
        display(Audio("error-126627.mp3" , autoplay=True))
        return [-1, -1]

    return (price, discount)

In [ ]:
def product_tracker(products, time_buffer):
    price_tracker = {}
    discount_tracker = {}
    time_tracker = []

    while True:
        time_tracker.append(0 if not time_tracker else time_tracker[-1] + time_buffer)

        for product, url in products.items():

            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            }

            session = requests.Session()
            response = session.get(url, headers=headers)

            if response.status_code != 200:
                print(f"Failed to retrieve the page. Status code: {response.status_code} for {product}")
                return

            soup = soup = BeautifulSoup(response.text, 'lxml')

            data = data_extractor(soup)

            if(data[0] == -1):
                print(f"Error handled for {product}")
                if product not in price_tracker.keys():
                    print(f"Error in the first request for {product} pls recheck the url")
                    return
                else:
                    price_tracker[product].append(price_tracker[product][-1])
                    discount_tracker[product].append(discount_tracker[product][-1])
            else:
                price_tracker.setdefault(product, []).append(data[0])
                discount_tracker.setdefault(product, []).append(data[1])

            if(len(price_tracker[product]) > 1 and price_tracker[product][-1] != price_tracker[product][-2]):
                display(Audio("military-alarm-129017.mp3" , autoplay=True))
                print(f"Price of {product} changed from {price_tracker[product][-2]} to {price_tracker[product][-1]}")


        display(Audio("interface-124464.mp3" , autoplay=True))

        fig, axs = plt.subplots(1, 2, figsize=(18, 4))

        for i in price_tracker.values():
            if(len(i) < 7):
                axs[0].plot(time_tracker, i, marker='o')
            else:
                axs[0].plot(time_tracker, i)

        axs[0].legend(price_tracker.keys())
        axs[0].grid(color='grey', linestyle='--', linewidth=0.5)
        axs[0].set_title('Prices Over Time')
        axs[0].set_xlabel("Time (min)")
        axs[0].set_ylabel("Price")

        for i in discount_tracker.values():
            if(len(i) < 7):
                axs[1].plot(time_tracker, i, marker='o')
            else:
                axs[1].plot(time_tracker, i)

        axs[1].legend(discount_tracker.keys())
        axs[1].grid(color='grey', linestyle='--', linewidth=0.5)
        axs[1].set_title('Discount Over Time')
        axs[1].set_xlabel("Time (min)")
        axs[1].set_ylabel("Discount")

        plt.show()

        time.sleep(60 * time_buffer)
        clear_output(wait=True)


In [ ]:
products = {"Apple iPhone 13 (128GB)": "https://www.amazon.in/Apple-iPhone-13-128GB-Product/dp/B09G99CW2N/ref=sr_1_3?crid=17AMWNAG7RZY3&keywords=Iphone&qid=1703069815&sprefix=iph%2Caps%2C221&sr=8-3",
            "Apple iPhone 15 Pro (512 GB)": "https://www.amazon.in/Apple-iPhone-15-Pro-512/dp/B0CHX7R6ZW/ref=sr_1_4?crid=17AMWNAG7RZY3&keywords=Iphone&qid=1703069815&sprefix=iph%2Caps%2C221&sr=8-4&th=1",
            "Apple iPhone 14 (256 GB)": "https://www.amazon.in/Apple-iPhone-256GB-Product-RED/dp/B0BDJ3R6QW/ref=sr_1_5?crid=17AMWNAG7RZY3&keywords=Iphone&qid=1703069815&sprefix=iph%2Caps%2C221&sr=8-5"}

try:
    product_tracker(products, 0.1)
except KeyboardInterrupt:
    print("Ending sequence")
except:
    print("something went wrong")